In [ ]:
# Importing libraries
import ee
from google.colab import drive
import geopandas as gpd
import geemap

In [ ]:
# Google Earth Engine Authentication
ee.Authenticate()
ee.Initialize(project='ee-lucasviturisantarosa')

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Define the area of interest (Uruguay)
aoi = ee.FeatureCollection('FAO/GAUL/2015/level0') \
         .filter(ee.Filter.eq('ADM0_NAME', 'Uruguay')) \
         .geometry()

aoi

# **GroundWater Storage**

https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V022_CLSM_G025_DA1D#description

In [ ]:
# Define the years of interest
anos = range(2003, 2023)  # Example: from 2003 to 2023

# Get geometry as coordinate list for export
region = aoi

# Define dataset and apply date filters
dataset = ee.ImageCollection('NASA/GLDAS/V022/CLSM/G025/DA1D') \
            .filter(ee.Filter.date('2003-01-01', '2023-12-31')) \
            .select('GWS_tavg')

# Function to calculate the daily mean
def calculate_daily_mean(ano, mes, dia):
    start = ee.Date.fromYMD(ano, mes, dia)
    end = start.advance(1, 'day')

    daily_images = dataset.filterDate(start, end)
    daily_mean = daily_images.mean().clip(aoi)

    return daily_mean.set('ano', ano).set('mes', mes).set('dia', dia)

# Function to export images to Google Drive
def export_image_to_drive(image, folder, description_prefix):
    ano = image.get('ano').getInfo()
    mes = image.get('mes').getInfo()
    dia = image.get('dia').getInfo()

    descricao = f'{description_prefix}_{ano:04d}_{mes:02d}_{dia:02d}'

    task = ee.batch.Export.image.toDrive(
        image=image,
        description=descricao,
        folder='TWS_UY',
        scale=25000,  # Adjust scale as needed
        region=region,
        maxPixels=1e13
    )
    task.start()
    print(f'Export started for: {descricao}')

# Iterate over years, months, and days, calculate daily mean, and export images
for ano in anos:
    for mes in range(1, 13):
        days_in_month = ee.Date.fromYMD(ano, mes, 1).advance(1, 'month').difference(ee.Date.fromYMD(ano, mes, 1), 'day').getInfo()
        for dia in range(1, int(days_in_month) + 1):
            mean_image = calculate_daily_mean(ano, mes, dia)
            export_image_to_drive(mean_image, 'GWS_UY', 'GWS_Diario_UY')

